In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#!conda install pytorch torchvision -c pytorch
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# necessary libraries
#import os
import random
import pandas as pd 
#import numpy as np 
import re
from spacy.tokens import Doc
import spacy
import torch
import torch.nn as nn
from spacy.tokenizer import Tokenizer
from tokenizers import BertWordPieceTokenizer
from torch.autograd import Variable
import urllib
from spacy.tokens import Doc
#from spacy.lang.en import English
from matplotlib import pyplot as plt  #matplotlib will be helpful to implement later to see if the training progresses.
from tensorflow.keras.preprocessing.sequence import pad_sequences   #padding was done in a loop- probably keras would be better for that, but i wasn't sure how it was implemented in it.
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras import regularizers
import tensorflow.keras.utils 
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertTokenizer   #Wasn't used.
import gensim
from gensim.test.utils import common_texts
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
w2v_model = Word2Vec(sentences = common_texts, vector_size = 100, window = 5, min_count = 1, workers = 4)
w2v_model.save("/kaggle/working/word2vec.model")

#os.makedirs('../word2vec')

MODEL_DIR = "/kaggle/input/huggingface-bert/"   #This wasn't used- it's for the vocabulary file for the bert tokenizer- but it has a lot of extra data.
#tokenizer = BertTokenizer(MODEL_DIR + "bert-large-uncased").encode()
#model = AutoModelForMaskedLM.from_pretrained(MODEL_DIR + "bert-large-uncased")
#model = Word2Vec(sentences, min_count=1)
#tokenizer = Word2Vec(sentences, min_count=1)
nlp = spacy.load('en_core_web_lg')

# loading the dataset
nlp_df = pd.read_csv(r'''../input/medium-articles/medium_articles.csv''')[:25]   #Loading first 100 medium articles (it's surprisingly a lot of data).
nlp_df.head()
nlp_df_texts = nlp_df['text']
nlp_df_texts.head()

In [ ]:
#general unit tests
#It's does the autotest of functions with the wrapper @autotest, the input is in the commented section- preceeded by ">>> ", the expected output is in the following line.
import doctest
import copy
import functools

def autotest(func):
    globs = copy.copy(globals())
    globs.update({func.__name__: func})
    doctest.run_docstring_examples(
        func, globs, verbose=True, name=func.__name__)
    return func

In [ ]:
#Copied from the course- setting the random seed.
# @title Set random seed

# @markdown Executing `set_seed(seed=seed)` you are setting the seed

# For DL its critical to set the random seed so that students can have a
# baseline to compare their results to expected results.
# Read more here: https://pytorch.org/docs/stable/notes/randomness.html

# Call `set_seed` function in the exercises to ensure reproducibility.
import random
import torch

def set_seed(seed=None, seed_torch=True):
  """
  Function that controls randomness. NumPy and random modules must be imported.

  Args:
    seed : Integer
      A non-negative integer that defines the random state. Default is `None`.
    seed_torch : Boolean
      If `True` sets the random seed for pytorch tensors, so pytorch module
      must be imported. Default is `True`.

  Returns:
    Nothing.
  """
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  if seed_torch:
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

  print(f'Random seed {seed} has been set.')


# In case that `DataLoader` is used
def seed_worker(worker_id):
  """
  DataLoader will reseed workers following randomness in
  multi-process data loading algorithm.

  Args:
    worker_id: integer
      ID of subprocess to seed. 0 means that
      the data will be loaded in the main process
      Refer: https://pytorch.org/docs/stable/data.html#data-loading-randomness for more details

  Returns:
    Nothing
  """
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

In [ ]:
#Just testing the loading through spacy.
doc = nlp("It's a simple sentence. Second simple sentence")
for token in doc:
    print(token)
#    print(type(token))
    print(token.text)
    print(type(token.text))
    token_str = "" + token.text
    print(type(token_str))

In [ ]:
# @title Set device (GPU or CPU). Execute `set_device()`
# especially if torch modules used.

# Copied from the course materials.
# Inform the user if the notebook uses GPU or CPU.
def set_device():
    '''
    Set the device. CUDA if available, CPU otherwise

    Args:
    None
    
    Returns:
    Nothing
    '''
#    device = "cuda" if torch.cuda.is_available() else "cpu"
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    if device != "cuda":
        print("WARNING: For this notebook to perform best, "
        "if possible, in the menu under `Runtime` -> "
        "`Change runtime type.`  select `GPU` ")
    else:
        print("GPU is enabled in this notebook.")

    return device

In [ ]:
SEED = 2021
set_seed(seed=SEED)
DEVICE = set_device()

In [ ]:
#Ruling out some stuff that I thought was causing errors. Ignore this cell.
str = 'abc'
if not str[-1] == 'b':
    str = str + 'c'
print(str)

In [ ]:
a = "abc"
b = ["a", "b", "c"]
c = [a, b]
print(type("abc"))
print(type(["a", "b", "c"]))
print(type(c[0]), type(c[1]), type(c))


In [ ]:
import tempfile
#This wasn't helpful and wasn't applied in the end.
with tempfile.NamedTemporaryFile(prefix='gensim-model-', delete=False) as tmp:
    temporary_filepath = tmp.name
    w2v_model.save(temporary_filepath)
    #
    # The model is now safely stored in the filepath.
    # You can copy it to other machines, share it with others, etc.
    #
    # To load a saved model:
    #
    new_model = gensim.models.Word2Vec.load(temporary_filepath)

In [ ]:
#Checking what sign could've been causing errors in the input. The error was caused by something else. Ignore this cell.
>>> '\x80'.encode().decode('windows-1252')

In [ ]:
#os.remove("/kaggle/working/word2vec.model")
#w2v_model = Word2Vec(sentences = common_texts, vector_size = 100, window = 5, min_count = 1, workers = 4)
#w2v_model.save("/kaggle/working/word2vec.model")
#DO NOT UNCOMMENT

In [ ]:
#data preprocessing and Tokenize
default_length = 50
#model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
# using gzipped/bz2 input works too, no need to unzip

@autotest
def get_sentences(article, autotest = 0, info = 0):
    '''
    Splits the article into sentences.
    
    Args: str (list elements from the csv of the dataset)
    
    Returns: list of str with "." being a delimiter.
    
    >>> get_sentences("First sentence. Second, sentence.", autotest = 1)
    ['First sentence.', 'Second,', 'sentence.']
    '''
    if not autotest:
        print("Splitting sentences.") if info else 0
    list_of_sentences = []
    temp_list_of_sentences = []
    for sentence in article.split(". "  or "! " or "; " or "? "):   #You could add other delimiters but for the sake of testing only the ". " was used.
        if sentence:
            sentence_copy = sentence
            if not sentence_copy[-1] == '.':   #.split() method removes the delimiter from the string- I'm adding that back because Word2Vec recognizes punctuation, too.
                sentence_copy = sentence_copy + '.'
            temp_list_of_sentences.append(sentence_copy)
    for sentence in temp_list_of_sentences:   #You could add other delimiters but for the sake of testing only the ". " was used.
        for subsentence in sentence.split(", " or "- "):
             if subsentence:
                sentence_copy = subsentence
                if not sentence_copy[-1] == ',' and not sentence_copy[-1] == ".":   #.split() method removes the delimiter from the string- I'm adding that back because Word2Vec recognizes punctuation, too.
                    sentence_copy = sentence_copy + ','
                if len(sentence_copy) > default_length:
                    for i in range(len(sentence_copy) // default_length):    
                        list_of_sentences.append(sentence_copy[i * default_length : (i + 1) * default_length - 1])
                    list_of_sentences.append(sentence_copy[(len(sentence_copy) // default_length) * default_length : ])
                else: 
                    list_of_sentences.append(sentence_copy)
                    
#        if len(list_of_sentences) >= 25:
#            print(list_of_sentences[-1])
    for sentence in list_of_sentences:
        sentence.replace("\n", " ")
        
#    if not autotest:
#        print(list_of_sentences)
    return list_of_sentences

@autotest
def train_dictionary(list_of_sentences, w2v_model = gensim.models.Word2Vec.load("/kaggle/working/word2vec.model"), autotest = 0, info = 0):
    '''
    Adds words to the dictionary.
    
    Args: a list of str, int (for the autotest)
    
    Returns: None (updates the dictionary file)
    
    train_dictionary([["First", "sentence", "sentence"], [0, 0, 0]], autotest = 1)
    [-0.00453036  0.11758299  0.00743674  0.22671193  0.02171193 -0.04433674
      0.02285469  0.11752532 -0.3050126   0.14662454 -0.19853342 -0.2143049
     -0.01861006 -0.07283057 -0.08528131 -0.18882579 -0.09247625  0.00860844
     -0.06703243  0.00666352 -0.05344337 -0.07154761 -0.02324829  0.02008165
     -0.03207938  0.03745614 -0.04832845 -0.07658006 -0.09805037 -0.08821845
      0.23835026  0.1279757   0.07754301 -0.15148504  0.00905246 -0.13989699
      0.12004489 -0.29544246 -0.00521144 -0.19496429 -0.06621867 -0.27184018
      0.04952901 -0.12884018 -0.152571   -0.19216733  0.11693359 -0.06423832
      0.21257243 -0.03538163  0.06508011 -0.23997843 -0.03135304  0.1233076
     -0.03994059  0.09518906  0.04058772  0.0460038  -0.12534082  0.10467226
      0.12010633  0.06119768 -0.01595154  0.0570786  -0.18848853 -0.08697353
     -0.13685709  0.00565706 -0.15765992  0.24080433 -0.0150929  -0.11440682
      0.19902213 -0.1956062   0.16332908 -0.046539   -0.05849491  0.23729242
      0.02069686  0.06861373 -0.04009774 -0.26628593 -0.2861018   0.07872369
     -0.11613636 -0.0366894   0.02865059  0.21707784  0.11488541 -0.01866605
      0.15172377 -0.01150565  0.021204    0.14768669  0.24422412  0.03802589
     -0.02380455 -0.21103993  0.00822991 -0.11126645]
    
    '''
    #total_examples=model.corpus_count, epochs=model.epochs
#    w2v_model = Word2Vec.load("word2vec.model")
    
#    if autotest:
#        print(w2v_model.wv['sentence'])
    if not autotest:
        print("Add words") if info else 0
    w2v_model.build_vocab(list_of_sentences, progress_per=10, update = True)   #Takes a list of sentences (list of str); if the str is long enough, there will be progress showing.  
    if not autotest:
        print("Train Word2Vec") if info else 0
#        print(list_of_sentences)
        print("Corpus total words: " + "{}".format(w2v_model.corpus_total_words)) if info else 0
#    w2v_model.train(list_of_sentences, total_examples = w2v_model.corpus_total_words, epochs = 2)
#    w2v_model = Word2Vec(list_of_sentences, min_count=1)
#    epochs = w2v_model.epochs
#    word_vectors = w2v_model.wv
#    word_vectors.save("word2vec.wordvectors")
    if not autotest:
        print("Save") if info else 0
    w2v_model.save("/kaggle/working/word2vec.model")
    if not autotest:
        print("Saved") if info else 0

    # Load back with memory-mapping = read-only, shared across processes.
    wv = gensim.models.Word2Vec.load("/kaggle/working/word2vec.model")
    if autotest:
        print(w2v_model.wv['sentence'])
    return w2v_model

@autotest
def tokenizer(sentences, autotest = 0, info = 0):
    '''
    Prepares final, vectorized (vectorization happens in a different function) token. It might need some tweaking to make it load unsplit sentences.
    
    Args: list of str
    
    Returns: list of lists of str (each str is a word, the inner lists correspond to sentences)
    
    tokenizer(["computer", "computer", 0], autotest = 1)
    [[-0.005157743580639362, -0.006670279428362846, -0.007779098581522703, 0.00831314641982317, -0.001982920104637742, -0.006856958847492933, -0.0041555981151759624, 0.0051456233486533165, -0.002869971562176943, -0.0037507526576519012, 0.0016218970995396376, -0.0027771019376814365, -0.0015848217299208045, 0.0010748032946139574, -0.0029788126703351736, 0.008521761745214462, 0.003912073094397783, -0.00996176153421402, 0.006261420901864767, -0.006756221409887075, 0.0007696555112488568, 0.004405516665428877, -0.005104861222207546, -0.00211128406226635, 0.008097834885120392, -0.004245028831064701, -0.00763848377391696, 0.009260606952011585, -0.0021561244502663612, -0.004720807541161776, 0.00857329461723566, 0.00428458396345377, 0.004326095338910818, 0.009287215769290924, -0.008455540984869003, 0.005256849806755781, 0.002039944985881448, 0.0041894991882145405, 0.0016983944224193692, 0.004465431906282902, 0.004487594589591026, 0.006106298882514238, -0.0032030295114964247, -0.004577061161398888, -0.00042664064676500857, 0.002534471219405532, -0.0032641179859638214, 0.006059480831027031, 0.004155338276177645, 0.007766852155327797, 0.002570020267739892, 0.00811904389411211, -0.0013876135926693678, 0.008080278523266315, 0.003718096762895584, -0.008049667812883854, -0.003934760577976704, -0.002472597872838378, 0.004894467070698738, -0.0008724128711037338, -0.002831732388585806, 0.007835987024009228, 0.00932561419904232, -0.0016153983306139708, -0.005160751286894083, -0.00470312824472785, -0.004847461357712746, -0.009605620987713337, 0.0013724195305258036, -0.004226145334541798, 0.00252744322642684, 0.0056161172688007355, -0.004067090339958668, -0.009599374607205391, 0.0015471477527171373, -0.006702072452753782, 0.002495900494977832, -0.003781732404604554, 0.0070804813876748085, 0.0006404069717973471, 0.0035619752015918493, -0.0027399309910833836, -0.0017110463231801987, 0.007655020337551832, 0.0014080877881497145, -0.005852151662111282, -0.007836776785552502, 0.0012330447789281607, 0.0064565083011984825, 0.005557966884225607, -0.00897966418415308, 0.008594664745032787, 0.004048154689371586, 0.007471777964383364, 0.009749171324074268, -0.0072917030192911625, -0.009042593650519848, 0.005837699398398399, 0.009393946267664433, 0.0035079459194093943], [-0.005157743580639362, -0.006670279428362846, -0.007779098581522703, 0.00831314641982317, -0.001982920104637742, -0.006856958847492933, -0.0041555981151759624, 0.0051456233486533165, -0.002869971562176943, -0.0037507526576519012, 0.0016218970995396376, -0.0027771019376814365, -0.0015848217299208045, 0.0010748032946139574, -0.0029788126703351736, 0.008521761745214462, 0.003912073094397783, -0.00996176153421402, 0.006261420901864767, -0.006756221409887075, 0.0007696555112488568, 0.004405516665428877, -0.005104861222207546, -0.00211128406226635, 0.008097834885120392, -0.004245028831064701, -0.00763848377391696, 0.009260606952011585, -0.0021561244502663612, -0.004720807541161776, 0.00857329461723566, 0.00428458396345377, 0.004326095338910818, 0.009287215769290924, -0.008455540984869003, 0.005256849806755781, 0.002039944985881448, 0.0041894991882145405, 0.0016983944224193692, 0.004465431906282902, 0.004487594589591026, 0.006106298882514238, -0.0032030295114964247, -0.004577061161398888, -0.00042664064676500857, 0.002534471219405532, -0.0032641179859638214, 0.006059480831027031, 0.004155338276177645, 0.007766852155327797, 0.002570020267739892, 0.00811904389411211, -0.0013876135926693678, 0.008080278523266315, 0.003718096762895584, -0.008049667812883854, -0.003934760577976704, -0.002472597872838378, 0.004894467070698738, -0.0008724128711037338, -0.002831732388585806, 0.007835987024009228, 0.00932561419904232, -0.0016153983306139708, -0.005160751286894083, -0.00470312824472785, -0.004847461357712746, -0.009605620987713337, 0.0013724195305258036, -0.004226145334541798, 0.00252744322642684, 0.0056161172688007355, -0.004067090339958668, -0.009599374607205391, 0.0015471477527171373, -0.006702072452753782, 0.002495900494977832, -0.003781732404604554, 0.0070804813876748085, 0.0006404069717973471, 0.0035619752015918493, -0.0027399309910833836, -0.0017110463231801987, 0.007655020337551832, 0.0014080877881497145, -0.005852151662111282, -0.007836776785552502, 0.0012330447789281607, 0.0064565083011984825, 0.005557966884225607, -0.00897966418415308, 0.008594664745032787, 0.004048154689371586, 0.007471777964383364, 0.009749171324074268, -0.0072917030192911625, -0.009042593650519848, 0.005837699398398399, 0.009393946267664433, 0.0035079459194093943], [-0.0005362272495403886, 0.00023643016174901277, 0.005103349685668945, 0.009009272791445255, -0.009302949532866478, -0.007116809021681547, 0.006458871532231569, 0.008972988463938236, -0.005015428178012371, -0.0037633730098605156, 0.007380504626780748, -0.0015334725612774491, -0.004536614287644625, 0.006554050371050835, -0.004860160406678915, -0.0018160176696255803, 0.002876579761505127, 0.000991873792372644, -0.008285215124487877, -0.009448818862438202, 0.0073117660358548164, 0.005070262122899294, 0.006757693365216255, 0.0007628655293956399, 0.006350889336317778, -0.0034053658600896597, -0.0009464025497436523, 0.005768573377281427, -0.00752163864672184, -0.003936104942113161, -0.007511582225561142, -0.0009300422389060259, 0.009538118727505207, -0.007319166790693998, -0.0023337698075920343, -0.0019377422286197543, 0.008077435195446014, -0.005930895917117596, 4.516124681686051e-05, -0.004753734916448593, -0.009603550657629967, 0.005007293075323105, -0.008759587071835995, -0.004391825292259455, -3.5099983506370336e-05, -0.00029618263943120837, -0.007661240175366402, 0.009614741429686546, 0.004982056561857462, 0.00923314318060875, -0.008157918229699135, 0.004495797213166952, -0.004137077368795872, 0.000824534916318953, 0.008498618379235268, -0.004462177865207195, 0.004517500288784504, -0.0067869615741074085, -0.0035484887193888426, 0.0093985078856349, -0.0015776539221405983, 0.00032137156813405454, -0.0041406298987567425, -0.007682688068598509, -0.0015080093871802092, 0.002469794824719429, -0.0008880281238816679, 0.0055336616933345795, -0.002742977114394307, 0.002260065171867609, 0.005455794278532267, 0.008345952257514, -0.001453740638680756, -0.009208142757415771, 0.004370551090687513, 0.0005717849708162248, 0.007441906724125147, -0.000813283899333328, -0.002638413803651929, -0.008753009140491486, -0.0008565568714402616, 0.0028265619184821844, 0.005401427857577801, 0.007052655331790447, -0.005703122820705175, 0.0018588185776025057, 0.006088862195611, -0.004798052366822958, -0.003107261611148715, 0.006797628477215767, 0.0016314744716510177, 0.00018991708930116147, 0.0034736371599137783, 0.00021777629444841295, 0.00961882621049881, 0.005060603842139244, -0.008917391300201416, -0.007041561417281628, 0.0009014558745548129, 0.006392533890902996]]
    '''
#    if not autotest:
#        print("Creating vectorized tokens.")
    if type(sentences) == type("string"):
        sentences = [sentences]
    words_vector = []
    for word in sentences:
        if not autotest:
            print(word) if info else 0
        print(type(word), word, word in w2v_model.wv.key_to_index.keys()) if info else 0
        if word in w2v_model.wv.key_to_index.keys():
            vector = w2v_model.wv[word]
            print(vector) if info else 0
            vector = vector.tolist()
            print(vector) if info else 0
            words_vector.append(vector)
#    if not autotest:
        print(words_vector[0]) if info else 0
        print("\n\n\n") if info else 0
    print(len(words_vector[0])) if info else 0
    return words_vector

#print(tokenizer(["computer"]))
padding_length = len(tokenizer(["computer"])[0])
print(padding_length)

print(np.zeros((1, padding_length)).tolist())
            
@autotest
def padding(sentence, max_sentence_length = default_length, labels = 0, info = 0):
    '''
    Fills in the spots in tokens with zeros until all the tokens have an equal length (default_length).
    padding(['It', "'s", 'a', 'simple', 'sentence', '.'])
    ['It', "'s", 'a', 'simple', 'sentence', '.', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '''
#    print(sentence.shape[0] < max(default_length, max_sentence_length))
#    while sentence.shape[0] < max(default_length, max_sentence_length):
#        if labels:
#            sentence[-1] = 0
#        sentence = np.append(sentence, [0])
    while len(sentence) < default_length:
        if labels:   #Conditional to remove one word from the sentence for the training set (replace with 0).
            sentence[-1] = (0 * np.array(sentence[0])).tolist()
#        print(sentence)
#        sentence += [(0 * np.array(sentence[0])).tolist()]
        sentence += np.zeros((1, padding_length)).tolist()
#        sentence[-1] = int(sentence[-1])
#        print(sentence)
    print(sentence) if info else 0
    return sentence
        
@autotest
def sentence_token(sentence, vectorized = 1, labels = 0, autotest = 0, info = 0, pad = 1):
    '''
    Creating tokens for each sentence.
    
    Args: str, int (0 or 1 whether to apply word2vec), int (0 or 1 whether it's for training or labels)
    
    Returns: a list of str
    
    sentence_token("It's a simple sentence.", 0, autotest = 1)
    ['It', "'s", 'a', 'simple', 'sentence', '.', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '''
    if not autotest:
        print("Creating sentence tokens.") if info else 0
#        print(sentence)
    if vectorized:
        sentence_token = []
    else:
        sentence_token = []
    doc = nlp(sentence)
    print(type(doc)) if info else 0
    index = 0
    for token in doc:
#        if vectorized:
#            sentence_token = torch.cat((sentence_token, torch.tensor([[Tokenizer(token)]])), dim = 1)
#        else:
#            sentence_token = torch.cat((sentence_token, token), dim = 1)
        if index == 0:
            print(tokenizer(token.text, autotest)) if info else 0
            print(token.text) if info else 0
        index += 1
        if vectorized:
            sentence_token += tokenizer(token.text, autotest)
#            sentence_token.append(tokenizer(token.text, autotest))

        else:
            sentence_token.append(token.text)
    
#    print(sentence_token.shape[0])
    if pad:
        sentence_token = padding(sentence_token, labels)
#    print(padding(sentence_token, training = 0 ))
    
    print(sentence_token) if info else 0
#    if vectorized:
#        flattened_token = torch.flatten(torch.tensor(sentence_token), start_dim = 0)    #Flattening to feature vectores of the size of 10000x1 (from 100x100) per sentence
#        print(flattened_token.shape) if info else 0
#        flattened_token = flattened_token.tolist()
#    else:
#        flattened_token = sentence_token
    
    return sentence_token
    
@autotest
def tokenize_article(article, vectorized = 1, labels = 0, autotest = 0, info = 0):
    '''
    Creates a list of lists of tokens of the article. The last sentence of the article retains a dot.
    
    Args: str from the dataset (a whole article), int (vectorization), int (training/labels)
    
    Returns: a list of lists containing str
    
    tokenize_article("First sentence. Second sentence.", 0, autotest = 1)
    [['First', 'sentence', '.', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ['Second', 'sentence', '.', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
    '''
    sentence_token_list = []
    for sentence in get_sentences(article, autotest):
#        if not autotest:
        print(sentence) if info else 0
#            print(sentence_token(sentence,vectorized, labels))
        sentence_token_list += [sentence_token(sentence,vectorized, labels, autotest, pad = 1)]
        print(sentence_token_list) if info else 0
#    if not autotest:
#        print("Article tokenized.")
#    print(torch.tensor(sentence_token_list).shape) if not info else 0
    return sentence_token_list

In [ ]:
w2v_model.wv.key_to_index.keys();

In [ ]:
dataset = nlp_df_texts   
def create_vocab(dataset, vectorized = 1, labels = 0, autotest = 0, info = 0):
    '''
    Split into sentences -> create sentence tokens -> apply word2vec to sentence tokens of the form: [["str", "str", ..., "."], ["str", "str", ..., "."], ...]
    
    Args: str (entire dataset)
    
    Returns: None (updates the vocabulary)
    '''
    w2v_model = gensim.models.Word2Vec.load("/kaggle/working/word2vec.model")
    for article in dataset:
        vocab_len = len(w2v_model.wv)
#        print("Vocabulary count before training step: " + "{}".format(vocab_len))
        w2v_model = gensim.models.Word2Vec.load("/kaggle/working/word2vec.model")
    
#        list_of_sentences = tokenize_article(article, 0)
        sentences = get_sentences(article)
        list_of_sentences = []
        for sentence in sentences:
            list_of_sentences += [sentence_token(sentence, vectorized = 0, labels = 0, autotest = 0, info = 0, pad = 0)]
#        print(list_of_sentences)
#        print(list_of_sentences)
        w2v_model = train_dictionary(list_of_sentences)
        w2v_model.save("/kaggle/working/word2vec.model")
        w2v_model = gensim.models.Word2Vec.load("/kaggle/working/word2vec.model")
        vocab_len = len(w2v_model.wv)
        print("Vocabulary count after training step: " + "{}".format(vocab_len), end = "\r")
#    w2v_model = KeyedVectors.load("word2vec.model", mmap='r')    
    print(w2v_model.wv.key_to_index.keys()) if info else 0
    return w2v_model
create_vocab(dataset, info = 0)  #Starts vocabulary training
w2v_model = gensim.models.Word2Vec.load("/kaggle/working/word2vec.model")
#w2v_model.wv.key_to_index.keys()

In [ ]:
string = "simple computer. very simple"
tempsentences = get_sentences(string)
print(tempsentences)
print(tempsentences[0])
toks = sentence_token(tempsentences[0], vectorized = 1)
#print(tokens)
#print(tokenizer(["simple"]))
tok = tokenize_article(string)
print(torch.tensor(toks).shape)
print(torch.tensor(tok).shape)
t = torch.tensor(tok)    #Need to flatten sentences in vectorized form, so each sentence goes from 100 x 100 to 10000 x 1.
#print(t.shape)
flattened = torch.flatten(t, start_dim = 1)
flattened.shape

In [ ]:
w2v_model = gensim.models.Word2Vec.load("/kaggle/working/word2vec.model")
w2v_model.wv.key_to_index.keys();

In [ ]:
# LSTM() returns tuple of (tensor, (recurrent state))
w2v_length = 100
class extract_tensor(nn.Module):
    def forward(self,x):
        # Output shape (batch, features, hidden)
        tensor, _ = x
        # Reshape shape (batch, hidden)
        return tensor[:, -1, :]


#RNN (LSTM)
dataset = nlp_df_texts
model = Word2Vec.load("word2vec.model")
class RNN_LSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, n_layers = 2, lr = 0.003, batches = 0):
        super(RNN_LSTM, self).__init__()
        self.embedding_dim = embedding_dim   #the dimensions of the input.
        self.hidden_dim = hidden_dim   #Width of the model.
        self.batches = batches    #Number of examples
        self.num_layers = n_layers   #Depth of the model.
        self.lr = lr   #Learning rate
        self.h_0 = torch.nn.Parameter(torch.rand(self.num_layers * 2, self.batches, int(0.5 * np.floor(self.hidden_dim)), requires_grad = True))
        self.c_0 = torch.nn.Parameter(torch.rand(self.num_layers * 2, self.batches, int(1 * np.floor(self.hidden_dim)), requires_grad = True))
        print(self.embedding_dim, self.hidden_dim, self.num_layers, self.lr)
        # Next line is for setting up the neural net, LSTM is the RNN variant (bidirectional).
        self.modnlp = nn.Sequential(
            nn.LSTM(input_size = w2v_length, hidden_size = self.hidden_dim, proj_size = int(np.floor(0.5 * self.hidden_dim)), num_layers = self.num_layers, batch_first = True, dropout = 0.3, bidirectional = True),
            extract_tensor(),
            nn.Linear(in_features = self.hidden_dim , out_features = int(np.floor(0.5 * self.embedding_dim)), bias = True , device = None , dtype = None ),
            nn.Softmax(dim = 1)
#            self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        )
        
    
        
    def load_article(self, article, vectorized = 1, labels = 0):   #Helper function.
#        print(article)
        loaded_article = tokenize_article(article, vectorized, labels)
#        print(loaded_article)
#        print(loaded_article)
        self.max_sentence_length = lambda max_sentence_length: max(article.map(len))
#        print(tokenize_article(article, vectorized, labels))
        return loaded_article
    
    def load_all_articles(self, dataset, vectorized = 1):   #Loading the dataset into the object.
        training_dataset = []
        labels_dataset = []
        index = 1
        for article in dataset:
            print("Article " + f"{index}", end = "\r")
#            print(article)
#            print(self.load_article(article, vectorized = 1, labels = 0))
            training_dataset += self.load_article(article, vectorized = 1, labels = 0)
            labels_dataset += self.load_article(article, vectorized = 1, labels = 1)
#            print(torch.tensor(training_dataset).shape)
            index += 1
        print(torch.tensor(labels_dataset).shape)
#        print(training_dataset)
        no_batches = torch.tensor(training_dataset).shape[0]
        print(no_batches)
        return (training_dataset, labels_dataset, no_batches)
    
    def forward(self, training_dataset, labels_dataset, epochs = 5):   #Training step.
        linear = nn.Linear(in_features = self.hidden_dim , out_features = int(np.floor(0.5 * self.embedding_dim)), bias = True , device = None , dtype = None )
        lstm = nn.LSTM(input_size = w2v_length, hidden_size = self.hidden_dim, proj_size = int(np.floor(0.5 * self.hidden_dim)), num_layers = self.num_layers, batch_first = True, dropout = 0.3, bidirectional = True)
        softmax = nn.Softmax(dim = 1)
        training_dataset = torch.tensor(training_dataset)
        labels_dataset = torch.tensor(labels_dataset)
        print(training_dataset.shape)
        print(type(training_dataset))
        loss_function = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), self.lr)
#        print(self.parameters())
        h_0 = self.h_0
        c_0 = self.c_0
        h_0.requires_grad_(True)
        c_0.requires_grad_(True)
        logits = torch.tensor(torch.zeros(1))
        logits.requires_grad_(True)
        self.train()
        for epoch in range(epochs):
            optimizer.zero_grad()
            logits, (h_n, c_n) = lstm(training_dataset, (h_0, c_0))
#            print(torch.tensor(logits).shape)
            logits = linear(logits)
            logits = softmax(logits)
            logits = torch.tensor(logits).requires_grad_(True)
#            print(torch.tensor(logits).shape)
            logits_labels = linear(labels_dataset)
            logits_labels = softmax(logits_labels)
            logits_labels = torch.tensor(logits_labels)
#            print(soft_logits[0] - soft_logits_labels[0])
            print(logits.shape)
            loss = loss_function(logits, logits_labels)
            loss.requires_grad_(True)
            loss.retain_grad()
            loss.backward(retain_graph = True)
            optimizer.step()
            print(loss.item(), "epoch" + f"{epoch}")
#            print(logits.grad)
        self.eval()
        return self.parameters, loss
        #self.modnlp(training_dataset, labels_dataset)
        
#        print(self.model.summary())

    def forward2(self, training_dataset, labels_dataset, epochs = 5):   #Training step.
        training_dataset = torch.tensor(training_dataset)
        labels_dataset = torch.tensor(labels_dataset)
        print(training_dataset.shape)
        print(type(training_dataset))
        loss_function = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), self.lr)
#        print(self.parameters())
        h_0 = self.h_0
        c_0 = self.c_0
        h_0.requires_grad_(True)
        c_0.requires_grad_(True)
        logits = torch.tensor(torch.zeros(1))
        logits.requires_grad_(True)
        self.train()
        for epoch in range(epochs):
            optimizer.zero_grad()
            logits = self.modnlp(training_dataset).requires_grad_(True)
#            print(torch.tensor(logits).shape)
            logits_labels = self.modnlp(labels_dataset)
#            print(soft_logits[0] - soft_logits_labels[0])
            print(logits.shape)
            loss = loss_function(logits, logits_labels)
            loss.requires_grad_(True)
            loss.retain_grad()
            loss.backward(retain_graph = True)
            optimizer.step()
            print(loss.item(), "epoch" + f"{epoch}")
#            print(logits.grad)
        self.eval()
        return self.parameters, loss

In [ ]:
print(dataset)
#test_article = tokenize_article(dataset[0], vectorized = 1, labels = 0);
#test_article2 = test_article[0]
#print(len(test_article[:][:][0][:]))
#print(test_article[:][:][0][:])

In [ ]:
training_dataset, labels_dataset, no_batches = RNN_LSTM(default_length, hidden_dim = w2v_length, n_layers = 2, lr = 0.003).load_all_articles(dataset)    #Loads the dataset.

In [ ]:
text_generator = RNN_LSTM(default_length, hidden_dim = w2v_length, batches = no_batches, n_layers = 2, lr = 0.003).forward2(training_dataset, labels_dataset, epochs = 20)   #Runs the model.

In [ ]:
#if nlp_df:
#    del nlp_df

In [ ]:
torch.tensor(training_dataset).shape